In [ ]:
#import project

Cloning into 'assignment3-21-robustnet-1'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 49 (delta 17), reused 23 (delta 5), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [ ]:
%cd /content/assignment3-21-robustnet-1/

/content/assignment3-21-robustnet-1


In [ ]:
!pip install -r requirements.txt

In [ ]:
#The attack is in the file carlini_wagner_l2.py
from carlini_wagner_l2 import (
   carlini_wagner_l2,
)

In [ ]:
#!/usr/bin/env python3 
import os
import argparse
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torchvision.transforms as transforms

from attacks import *

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

valid_size = 1024 
batch_size = 32 
criterion = nn.NLLLoss()
'''Basic neural network architecture (from pytorch doc).'''
class Net(nn.Module):

    model_file="models/default_model.pth"
    '''This file will be loaded to test your model. Use --model-file to load/store a different model.'''
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)
        return x
    

    

    def save(self, model_file):
        '''Helper function, use it to save the model weights after training.'''
        torch.save(self.state_dict(), model_file)

    def load(self, model_file):
        self.load_state_dict(torch.load(model_file, map_location=torch.device(device)))

        
    def load_for_testing(self, project_dir='./'):
        '''This function will be called automatically before testing your
           project, and will load the model weights from the file
           specify in Net.model_file.
           
           You must not change the prototype of this function. You may
           add extra code in its body if you feel it is necessary, but
           beware that paths of files used in this function should be
           refered relative to the root of your project directory.
        '''        
        self.load(os.path.join(project_dir, Net.model_file))




####### Testing ############
def test_natural(net, test_loader):
    '''Basic testing function.'''

    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for i,data in enumerate(test_loader, 0):
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total
    
######################## Attack Testing ########################


def test_carlini_wagner(net,test_loader):
  correct = 0
  for i,data in enumerate(test_loader, 0):
            images, labels = data[0].to(device), data[1].to(device)
            adv_data = images.clone().detach().to(device)
            # calculate outputs by running images through the network
            adv_data.requires_grad = True
            outputs = net(adv_data)
            # the class with the highest energy is what we choose as prediction
            _, nat_pred = torch.max(outputs.data, 1)
            if nat_pred.item() != labels.item():
              continue
            #if test sample is correctly predicted
            loss = criterion(outputs, labels)
            net.zero_grad()
            loss.backward()

            data_grad = adv_data.grad.data
            adv_data = carlini_wagner_l2(net,images,10,targeted=False,y=nat_pred)
            output = net(adv_data)
            _,adv_pred = torch.max(output.data, 1)
            if adv_pred.item() == labels.item():
              correct+=1
            print("Test num "+str(i))
  final_acc = correct/float(len(test_loader))
  print("\tTest Accuracy = {} / {} = {}".format( correct, len(test_loader), final_acc))
  return 100*final_acc



##########################################

def get_train_loader(dataset, valid_size=1024, batch_size=32):
    '''Split dataset into [train:valid] and return a DataLoader for the training part.'''

    indices = list(range(len(dataset)))
    train_sampler = torch.utils.data.SubsetRandomSampler(indices[valid_size:])
    train = torch.utils.data.DataLoader(dataset, sampler=train_sampler, batch_size=batch_size)

    return train

def get_validation_loader(dataset, valid_size=1024, batch_size=32):
    '''Split dataset into [train:valid] and return a DataLoader for the validation part.'''

    indices = list(range(len(dataset)))
    valid_sampler = torch.utils.data.SubsetRandomSampler(indices[:valid_size])
    valid = torch.utils.data.DataLoader(dataset, sampler=valid_sampler, batch_size=batch_size)

    return valid
net = Net()
net.to(device)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
cifar = torchvision.datasets.CIFAR10('./data/', download=True, transform=transforms.ToTensor())
valid_loader = get_validation_loader(cifar, valid_size, 1)  # batch size one for attack test otherwise put 32
net.load("models/distilled_model.pth")
acc = test_natural(net, valid_loader)
print("Model natural accuracy (valid): {}".format(acc))


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data/
Model natural accuracy (valid): 60.15625


In [ ]:
acc = test_carlini_wagner(net, valid_loader)

Test num 1 label:2 predicted 4
Test num 2 label:9 predicted 1
Test num 4 label:6 predicted 4
Test num 5 label:6 predicted 3
Test num 6 label:1 predicted 0
Test num 8 label:3 predicted 4
Test num 9 label:5 predicted 4
Test num 10 label:2 predicted 6
Test num 11 label:5 predicted 3
Test num 12 label:9 predicted 1
Test num 14 label:7 predicted 5
Test num 16 label:2 predicted 5
Test num 18 label:4 predicted 6
Test num 19 label:1 predicted 8
Test num 20 label:7 predicted 4
Test num 21 label:5 predicted 3
Test num 23 label:1 predicted 2
Test num 24 label:1 predicted 9
Test num 25 label:9 predicted 3
Test num 26 label:5 predicted 2
Test num 28 label:0 predicted 8
Test num 29 label:8 predicted 9
Test num 31 label:4 predicted 2
Test num 33 label:8 predicted 1
Test num 38 label:2 predicted 0
Test num 40 label:6 predicted 2
Test num 41 label:6 predicted 8
Test num 42 label:1 predicted 9
Test num 43 label:9 predicted 8
Test num 44 label:2 predicted 4
Test num 45 label:1 predicted 9
Test num 46 lab